In [38]:
import numpy as np
import matplotlib.pyplot as plt
import yaml
import os
import pandas as pd

root = './experiments/lr_batch_20241109/results/'
files = os.listdir(root)



In [2]:
files[0]

'pytorch_classifier_mnist_50_trial=8_results.yaml'

In [12]:
with open(os.path.join(root, files[0]), "r") as f:
    res = yaml.safe_load(f)

In [17]:
files[0]

'pytorch_classifier_mnist_50_trial=8_results.yaml'

In [35]:
def get_best_lrs(root):
    results = []
    for file in os.listdir(root):
        with open(os.path.join(root, file), "r") as f:
            res = yaml.safe_load(f)
        best_metrics = res['best_metrics']
        results.append({
            'lr': best_metrics['config']['train_loop_config']['optimizer_cfg']['args']['lr'],
            'epoch': best_metrics['epoch'],
            'val_loss': best_metrics['val_loss'],
            'split_perc': int(file.split("pytorch_classifier_mnist_")[1].split("_trial")[0])})
    return results
        

In [36]:
results = get_best_lrs(root)

In [40]:
results_df = pd.DataFrame(results)

In [41]:
results_df

,lr,epoch,val_loss,split_perc
0,0.000782,2,0.009042,50
1,0.060619,1,0.010575,75
2,0.000003,2,0.012808,25
3,0.000009,2,0.007676,100
4,0.000008,2,0.007463,100
5,0.000002,5,0.009345,50
6,0.000007,2,0.007597,100
7,0.000518,2,0.009342,50
8,0.004300,2,0.018835,10
9,0.000008,2,0.012695,25
